In [1]:
years = [2021, 2022, 2023]
months = [f"0{x}" for x in range(1,10)] + ["10", "11", "12"]

# Strategy

### 1. Predict demand of each zone without aggregation
Covariates:

a) Average total fair time series for each zone  
b) Demand time series from top 2 nearest zones  
c) Temperature time series for each zone  
d) Precipitation time series for each zone

### 2. Predict on each aggregation and reconcile to its child zones

##### Point of Interest

School, Pri School, Sec School, University, Food, Food Bar, Food Morning, Food Restaurants, Attractions, Religious

Covariates:

a) Average total fair time series of each aggregation  
b) Aggregated demand time series of the 2 nearest zones for each child zone  

##### Geography

Financial District, etc

Covariates:

a) Average total fair time series of each aggregation  
b) Temperature time series for each aggregation  
c) Precipitation time series for each aggregation

##### Total number of cars in zones

Each zone will have an average number of combined total number of cars for each hour. 
We can aggregate them based on: 
- zones which have similar congestion for each time of day
- zones with have similar weekday daily traffic

Covariates:

a) Average total fair time series of each aggregation  
b) Aggregated demand time series of the 2 nearest zones for each child zone   


##### Total Population

We can aggregate areas with similar populations (based on 2010 data)

Covariates:

a) Average total fair time series of each aggregation  
b) Aggregated demand time series of the 2 nearest zones for each child zone   

# Total demand time series

In [2]:
# Combining demand time series
# initialise first df as 2021 Jan
import pandas as pd

combined_df = pd.read_csv(f'Time_Series_CSVs/2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_CSVs/{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_df = pd.concat([combined_df, df], axis=0, ignore_index=True)

combined_df

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,1.0,0.0,1.0,2.0,2.0,0.0,6.0,1.0,20.0,...,5.0,3.0,5.0,0.0,0.0,2.0,5.0,1.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,0.0,1.0,2.0,2.0,2.0,8.0,2.0,20.0,...,11.0,11.0,5.0,0.0,0.0,2.0,1.0,1.0,2.0,16.0
2,2021-01-01 00:30:00,1.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,21.0,...,10.0,9.0,6.0,0.0,2.0,3.0,6.0,0.0,4.0,14.0
3,2021-01-01 00:45:00,0.0,0.0,1.0,3.0,0.0,0.0,5.0,0.0,20.0,...,9.0,16.0,12.0,0.0,0.0,2.0,8.0,0.0,8.0,17.0
4,2021-01-01 01:00:00,0.0,0.0,0.0,1.0,9.0,5.0,1.0,0.0,18.0,...,4.0,7.0,15.0,0.0,0.0,2.0,12.0,1.0,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,0.0,5.0,1.0,3.0,0.0,9.0,1.0,52.0,...,34.0,6.0,20.0,0.0,0.0,12.0,60.0,4.0,3.0,15.0
99248,2023-10-31 23:00:00,3.0,0.0,3.0,1.0,1.0,0.0,11.0,4.0,34.0,...,28.0,6.0,9.0,0.0,0.0,13.0,65.0,3.0,6.0,15.0
99249,2023-10-31 23:15:00,0.0,0.0,1.0,1.0,0.0,2.0,10.0,1.0,39.0,...,23.0,7.0,14.0,0.0,1.0,10.0,68.0,1.0,3.0,15.0
99250,2023-10-31 23:30:00,1.0,1.0,3.0,0.0,1.0,1.0,5.0,4.0,44.0,...,31.0,7.0,15.0,0.0,1.0,16.0,76.0,1.0,2.0,9.0


In [3]:
# Get the memory usage of the DataFrame
memory_usage = combined_df.memory_usage(deep=True).sum()
print(f"Memory usage of the DataFrame: {memory_usage} bytes")

Memory usage of the DataFrame: 59154324 bytes


In [4]:
combined_df.to_csv('combined_demand_df.csv', index=False, header=True)

In [5]:
a = pd.read_csv('combined_demand_df.csv')
a

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,1.0,0.0,1.0,2.0,2.0,0.0,6.0,1.0,20.0,...,5.0,3.0,5.0,0.0,0.0,2.0,5.0,1.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,0.0,1.0,2.0,2.0,2.0,8.0,2.0,20.0,...,11.0,11.0,5.0,0.0,0.0,2.0,1.0,1.0,2.0,16.0
2,2021-01-01 00:30:00,1.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,21.0,...,10.0,9.0,6.0,0.0,2.0,3.0,6.0,0.0,4.0,14.0
3,2021-01-01 00:45:00,0.0,0.0,1.0,3.0,0.0,0.0,5.0,0.0,20.0,...,9.0,16.0,12.0,0.0,0.0,2.0,8.0,0.0,8.0,17.0
4,2021-01-01 01:00:00,0.0,0.0,0.0,1.0,9.0,5.0,1.0,0.0,18.0,...,4.0,7.0,15.0,0.0,0.0,2.0,12.0,1.0,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,0.0,5.0,1.0,3.0,0.0,9.0,1.0,52.0,...,34.0,6.0,20.0,0.0,0.0,12.0,60.0,4.0,3.0,15.0
99248,2023-10-31 23:00:00,3.0,0.0,3.0,1.0,1.0,0.0,11.0,4.0,34.0,...,28.0,6.0,9.0,0.0,0.0,13.0,65.0,3.0,6.0,15.0
99249,2023-10-31 23:15:00,0.0,0.0,1.0,1.0,0.0,2.0,10.0,1.0,39.0,...,23.0,7.0,14.0,0.0,1.0,10.0,68.0,1.0,3.0,15.0
99250,2023-10-31 23:30:00,1.0,1.0,3.0,0.0,1.0,1.0,5.0,4.0,44.0,...,31.0,7.0,15.0,0.0,1.0,16.0,76.0,1.0,2.0,9.0


# Total supply time series

In [11]:
# Combining demand time series
# initialise first df as 2021 Jan
import pandas as pd

combined_df_supply = pd.read_csv(f'Time_Series_CSVs_Supply/2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_CSVs_Supply/{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_df_supply = pd.concat([combined_df_supply, df], axis=0, ignore_index=True)

combined_df_supply

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,2.0,0.0,1.0,0.0,4.0,1.0,1.0,0.0,12.0,...,4.0,7.0,9.0,0.0,1.0,1.0,1.0,1.0,4.0,4.0
1,2021-01-01 00:15:00,4.0,0.0,1.0,5.0,5.0,2.0,3.0,1.0,7.0,...,10.0,13.0,9.0,0.0,1.0,0.0,2.0,3.0,7.0,9.0
2,2021-01-01 00:30:00,1.0,0.0,1.0,3.0,2.0,3.0,8.0,0.0,14.0,...,6.0,13.0,8.0,1.0,1.0,1.0,1.0,4.0,6.0,10.0
3,2021-01-01 00:45:00,1.0,0.0,0.0,3.0,10.0,5.0,3.0,0.0,11.0,...,7.0,13.0,11.0,0.0,0.0,4.0,8.0,2.0,6.0,13.0
4,2021-01-01 01:00:00,5.0,0.0,3.0,1.0,3.0,10.0,3.0,0.0,8.0,...,4.0,9.0,10.0,0.0,2.0,2.0,6.0,1.0,7.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99246,2023-10-31 22:45:00,8.0,0.0,9.0,3.0,5.0,7.0,3.0,4.0,47.0,...,21.0,32.0,19.0,3.0,8.0,22.0,22.0,4.0,25.0,30.0
99247,2023-10-31 23:00:00,7.0,0.0,11.0,2.0,8.0,8.0,5.0,3.0,38.0,...,24.0,23.0,27.0,4.0,6.0,26.0,20.0,12.0,13.0,15.0
99248,2023-10-31 23:15:00,8.0,0.0,9.0,4.0,6.0,6.0,3.0,3.0,34.0,...,28.0,26.0,25.0,6.0,7.0,13.0,16.0,3.0,20.0,30.0
99249,2023-10-31 23:30:00,10.0,0.0,4.0,3.0,4.0,10.0,3.0,4.0,34.0,...,20.0,16.0,16.0,5.0,5.0,21.0,27.0,6.0,6.0,24.0


In [12]:
# Get the memory usage of the DataFrame
memory_usage = combined_df_supply.memory_usage(deep=True).sum()
print(f"Memory usage of the DataFrame: {memory_usage} bytes")

Memory usage of the DataFrame: 59153728 bytes


In [13]:
combined_df_supply.to_csv('combined_supply_df.csv', index=False, header=True)

In [14]:
b = pd.read_csv('combined_supply_df.csv')
b

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,2.0,0.0,1.0,0.0,4.0,1.0,1.0,0.0,12.0,...,4.0,7.0,9.0,0.0,1.0,1.0,1.0,1.0,4.0,4.0
1,2021-01-01 00:15:00,4.0,0.0,1.0,5.0,5.0,2.0,3.0,1.0,7.0,...,10.0,13.0,9.0,0.0,1.0,0.0,2.0,3.0,7.0,9.0
2,2021-01-01 00:30:00,1.0,0.0,1.0,3.0,2.0,3.0,8.0,0.0,14.0,...,6.0,13.0,8.0,1.0,1.0,1.0,1.0,4.0,6.0,10.0
3,2021-01-01 00:45:00,1.0,0.0,0.0,3.0,10.0,5.0,3.0,0.0,11.0,...,7.0,13.0,11.0,0.0,0.0,4.0,8.0,2.0,6.0,13.0
4,2021-01-01 01:00:00,5.0,0.0,3.0,1.0,3.0,10.0,3.0,0.0,8.0,...,4.0,9.0,10.0,0.0,2.0,2.0,6.0,1.0,7.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99246,2023-10-31 22:45:00,8.0,0.0,9.0,3.0,5.0,7.0,3.0,4.0,47.0,...,21.0,32.0,19.0,3.0,8.0,22.0,22.0,4.0,25.0,30.0
99247,2023-10-31 23:00:00,7.0,0.0,11.0,2.0,8.0,8.0,5.0,3.0,38.0,...,24.0,23.0,27.0,4.0,6.0,26.0,20.0,12.0,13.0,15.0
99248,2023-10-31 23:15:00,8.0,0.0,9.0,4.0,6.0,6.0,3.0,3.0,34.0,...,28.0,26.0,25.0,6.0,7.0,13.0,16.0,3.0,20.0,30.0
99249,2023-10-31 23:30:00,10.0,0.0,4.0,3.0,4.0,10.0,3.0,4.0,34.0,...,20.0,16.0,16.0,5.0,5.0,21.0,27.0,6.0,6.0,24.0


# Total demand time series for 1h time step

In [3]:
# Combining demand time series
# initialise first df as 2021 Jan
import pandas as pd

combined_df_1h = pd.read_csv(f'Time_Series_CSVs/1h_2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_CSVs/1h_{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_df_1h = pd.concat([combined_df_1h, df], axis=0, ignore_index=True)

combined_df_1h

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,2021-01-01 01:00:00,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2021-01-01 02:00:00,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,2021-01-01 03:00:00,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,2021-01-01 04:00:00,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,2023-10-31 20:00:00,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,2023-10-31 21:00:00,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,2023-10-31 22:00:00,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [4]:
combined_df_1h.to_csv('combined_demand_df_1h.csv', index=False, header=True)

In [5]:
a = pd.read_csv('combined_demand_df_1h.csv')
a

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,2021-01-01 01:00:00,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2021-01-01 02:00:00,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,2021-01-01 03:00:00,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,2021-01-01 04:00:00,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,2023-10-31 20:00:00,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,2023-10-31 21:00:00,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,2023-10-31 22:00:00,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


# Total supply time series for 1h time step

In [6]:
# Combining demand time series
# initialise first df as 2021 Jan
import pandas as pd

combined_df_supply_1h = pd.read_csv(f'Time_Series_CSVs_Supply/1h_2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_CSVs_Supply/1h_{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_df_supply_1h = pd.concat([combined_df_supply_1h, df], axis=0, ignore_index=True)

combined_df_supply_1h

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,8.0,0.0,3.0,11.0,21.0,11.0,15.0,1.0,44.0,...,27.0,46.0,37.0,1.0,3.0,6.0,12.0,10.0,23.0,36.0
1,2021-01-01 01:00:00,13.0,0.0,7.0,10.0,26.0,21.0,7.0,5.0,39.0,...,13.0,42.0,37.0,2.0,12.0,10.0,26.0,3.0,30.0,37.0
2,2021-01-01 02:00:00,9.0,1.0,4.0,5.0,10.0,13.0,3.0,4.0,43.0,...,11.0,21.0,21.0,8.0,12.0,10.0,12.0,2.0,16.0,32.0
3,2021-01-01 03:00:00,5.0,0.0,0.0,1.0,6.0,9.0,0.0,0.0,19.0,...,1.0,10.0,8.0,1.0,6.0,3.0,9.0,5.0,7.0,15.0
4,2021-01-01 04:00:00,2.0,0.0,0.0,3.0,9.0,10.0,0.0,1.0,11.0,...,3.0,9.0,1.0,0.0,1.0,2.0,7.0,1.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,40.0,1.0,33.0,32.0,60.0,32.0,59.0,13.0,163.0,...,316.0,207.0,221.0,7.0,11.0,97.0,143.0,17.0,142.0,153.0
24809,2023-10-31 20:00:00,22.0,0.0,28.0,36.0,50.0,21.0,70.0,13.0,150.0,...,257.0,190.0,217.0,5.0,12.0,105.0,89.0,25.0,154.0,135.0
24810,2023-10-31 21:00:00,29.0,0.0,40.0,21.0,41.0,27.0,52.0,15.0,154.0,...,176.0,178.0,204.0,15.0,18.0,93.0,76.0,29.0,111.0,119.0
24811,2023-10-31 22:00:00,35.0,1.0,41.0,22.0,29.0,30.0,18.0,11.0,185.0,...,125.0,134.0,121.0,14.0,22.0,93.0,87.0,22.0,96.0,119.0


In [7]:
combined_df_supply_1h.to_csv('combined_supply_df_1h.csv', index=False, header=True)

In [8]:
a = pd.read_csv('combined_supply_df_1h.csv')
a

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,8.0,0.0,3.0,11.0,21.0,11.0,15.0,1.0,44.0,...,27.0,46.0,37.0,1.0,3.0,6.0,12.0,10.0,23.0,36.0
1,2021-01-01 01:00:00,13.0,0.0,7.0,10.0,26.0,21.0,7.0,5.0,39.0,...,13.0,42.0,37.0,2.0,12.0,10.0,26.0,3.0,30.0,37.0
2,2021-01-01 02:00:00,9.0,1.0,4.0,5.0,10.0,13.0,3.0,4.0,43.0,...,11.0,21.0,21.0,8.0,12.0,10.0,12.0,2.0,16.0,32.0
3,2021-01-01 03:00:00,5.0,0.0,0.0,1.0,6.0,9.0,0.0,0.0,19.0,...,1.0,10.0,8.0,1.0,6.0,3.0,9.0,5.0,7.0,15.0
4,2021-01-01 04:00:00,2.0,0.0,0.0,3.0,9.0,10.0,0.0,1.0,11.0,...,3.0,9.0,1.0,0.0,1.0,2.0,7.0,1.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,40.0,1.0,33.0,32.0,60.0,32.0,59.0,13.0,163.0,...,316.0,207.0,221.0,7.0,11.0,97.0,143.0,17.0,142.0,153.0
24809,2023-10-31 20:00:00,22.0,0.0,28.0,36.0,50.0,21.0,70.0,13.0,150.0,...,257.0,190.0,217.0,5.0,12.0,105.0,89.0,25.0,154.0,135.0
24810,2023-10-31 21:00:00,29.0,0.0,40.0,21.0,41.0,27.0,52.0,15.0,154.0,...,176.0,178.0,204.0,15.0,18.0,93.0,76.0,29.0,111.0,119.0
24811,2023-10-31 22:00:00,35.0,1.0,41.0,22.0,29.0,30.0,18.0,11.0,185.0,...,125.0,134.0,121.0,14.0,22.0,93.0,87.0,22.0,96.0,119.0


### Covariate 1: Ave total fair time series for each location

In [3]:
combined_total_fair_df = pd.read_csv(f'Time_Series_Ave_Total_Fair_CSVs/2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_Ave_Total_Fair_CSVs/{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_total_fair_df = pd.concat([combined_total_fair_df, df], axis=0, ignore_index=True)

combined_total_fair_df

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,22.800000,0.0,28.300000,16.87,19.205000,0.000,13.385000,30.36,15.230000,...,12.824000,13.790000,12.522000,0.0,0.00,14.050000,22.582000,25.300000,8.300000,7.071429
1,2021-01-01 00:15:00,27.800000,0.0,23.800000,14.60,20.865000,31.050,20.066250,0.00,17.816000,...,14.746364,14.593636,12.618000,0.0,0.00,11.155000,12.800000,23.160000,18.975000,10.588750
2,2021-01-01 00:30:00,13.800000,0.0,0.000000,0.00,14.650000,0.000,13.386667,63.42,15.241429,...,11.848000,14.180000,14.355000,0.0,18.17,17.150000,16.703333,0.000000,13.850000,14.023571
3,2021-01-01 00:45:00,0.000000,0.0,21.800000,19.20,0.000000,0.000,14.246000,0.00,19.137500,...,12.984444,17.263125,15.713333,0.0,0.00,11.840000,23.570000,0.000000,14.260000,14.249412
4,2021-01-01 01:00:00,0.000000,0.0,0.000000,14.80,24.674444,18.000,12.960000,0.00,13.168889,...,21.685000,15.618571,13.231333,0.0,0.00,15.960000,22.605833,39.300000,9.584000,17.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,0.0,43.092000,21.70,11.453333,0.000,18.813333,14.64,23.352308,...,18.625588,22.686667,16.312500,0.0,0.00,24.539167,26.023167,35.407500,20.640000,13.883333
99248,2023-10-31 23:00:00,20.256667,0.0,32.426667,23.70,35.050000,0.000,21.124545,35.65,16.966176,...,18.188571,27.188333,22.350000,0.0,0.00,15.223077,27.037846,32.003333,15.461667,20.020667
99249,2023-10-31 23:15:00,0.000000,0.0,31.440000,13.60,0.000000,31.085,16.604000,18.70,22.244615,...,22.390435,22.872857,21.289286,0.0,22.55,22.104000,26.830588,18.840000,12.190000,17.386000
99250,2023-10-31 23:30:00,66.880000,26.4,19.006667,0.00,35.640000,11.100,19.174000,22.12,23.023409,...,22.044839,17.165714,19.174667,0.0,43.20,25.200000,25.789605,46.560000,14.585000,20.304444


In [9]:
combined_total_fair_df_1h = pd.read_csv(f'Time_Series_Ave_Total_Fair_CSVs/1h_2021/2021_01_Time_Series.csv')

for year in years:
    for month in months:
        if year == 2023 and month == "11":
            continue
        if year == 2023 and month == "12":
            continue
        else:
            df = pd.read_csv(f'Time_Series_Ave_Total_Fair_CSVs/1h_{year}/{year}_{month}_Time_Series.csv')
        if year == 2021 and month == "01":
            continue
        else:
            combined_total_fair_df_1h = pd.concat([combined_total_fair_df_1h, df], axis=0, ignore_index=True)

combined_total_fair_df_1h

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,21.466667,0.0,24.633333,17.220000,18.958000,31.050000,16.010455,23.445000,16.836296,...,13.190571,15.531538,14.299643,0.00,18.170000,13.948889,20.724500,24.230000,14.382000,12.175741
1,2021-01-01 01:00:00,11.300000,0.0,25.380000,13.975000,20.332727,18.222143,18.420000,0.000000,15.869167,...,19.783750,20.721481,15.215000,17.30,22.673333,18.970000,22.261563,26.596667,11.111111,15.520612
2,2021-01-01 02:00:00,12.300000,0.0,15.230000,22.070000,25.960909,14.662222,10.562000,0.000000,19.650476,...,15.229412,16.193043,17.193333,0.00,19.133333,22.927692,17.978824,19.500000,18.902857,14.780741
3,2021-01-01 03:00:00,14.366667,0.0,0.000000,18.450000,32.006667,19.637143,19.280000,23.300000,16.536129,...,11.364286,15.855000,18.052500,0.00,6.800000,23.370000,14.929000,15.300000,18.070000,19.595833
4,2021-01-01 04:00:00,0.000000,0.0,0.000000,26.300000,6.350000,13.140000,0.000000,0.000000,17.414500,...,14.550000,23.000000,14.081667,0.00,10.800000,14.940000,19.573333,20.625000,9.880000,22.089000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,31.550000,0.0,28.587568,16.588333,17.310741,22.100000,21.543529,32.169167,23.051000,...,21.788112,20.390838,20.162161,7.00,0.000000,24.597905,28.974898,32.943913,21.268000,20.502778
24809,2023-10-31 20:00:00,34.500000,0.0,31.561892,18.169000,21.064167,7.600000,20.582045,27.567143,21.364687,...,18.444010,19.204370,20.233822,52.09,41.330000,22.875806,26.767233,27.448077,19.635821,19.894821
24810,2023-10-31 21:00:00,29.730000,0.0,27.209630,14.012000,18.277500,18.075000,21.498163,20.048571,21.667453,...,19.601459,22.618684,18.950682,0.00,0.000000,24.917614,28.554130,30.371875,20.090000,19.403636
24811,2023-10-31 22:00:00,31.780000,0.0,32.099286,27.931111,9.445833,23.772500,20.993243,20.301818,23.467257,...,19.009235,22.166389,19.928830,0.00,0.000000,19.226604,26.789421,30.670833,18.930952,19.634500


In [10]:
combined_total_fair_df_1h.to_csv('combined_total_fair_df_1h.csv', index=False, header=True)

In [11]:
a = pd.read_csv('combined_total_fair_df_1h.csv')
a

,time_slot,4,12,13,24,41,42,43,45,48,...,237,238,239,243,244,246,249,261,262,263
0,2021-01-01 00:00:00,21.466667,0.0,24.633333,17.220000,18.958000,31.050000,16.010455,23.445000,16.836296,...,13.190571,15.531538,14.299643,0.00,18.170000,13.948889,20.724500,24.230000,14.382000,12.175741
1,2021-01-01 01:00:00,11.300000,0.0,25.380000,13.975000,20.332727,18.222143,18.420000,0.000000,15.869167,...,19.783750,20.721481,15.215000,17.30,22.673333,18.970000,22.261563,26.596667,11.111111,15.520612
2,2021-01-01 02:00:00,12.300000,0.0,15.230000,22.070000,25.960909,14.662222,10.562000,0.000000,19.650476,...,15.229412,16.193043,17.193333,0.00,19.133333,22.927692,17.978824,19.500000,18.902857,14.780741
3,2021-01-01 03:00:00,14.366667,0.0,0.000000,18.450000,32.006667,19.637143,19.280000,23.300000,16.536129,...,11.364286,15.855000,18.052500,0.00,6.800000,23.370000,14.929000,15.300000,18.070000,19.595833
4,2021-01-01 04:00:00,0.000000,0.0,0.000000,26.300000,6.350000,13.140000,0.000000,0.000000,17.414500,...,14.550000,23.000000,14.081667,0.00,10.800000,14.940000,19.573333,20.625000,9.880000,22.089000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2023-10-31 19:00:00,31.550000,0.0,28.587568,16.588333,17.310741,22.100000,21.543529,32.169167,23.051000,...,21.788112,20.390838,20.162161,7.00,0.000000,24.597905,28.974898,32.943913,21.268000,20.502778
24809,2023-10-31 20:00:00,34.500000,0.0,31.561892,18.169000,21.064167,7.600000,20.582045,27.567143,21.364687,...,18.444010,19.204370,20.233822,52.09,41.330000,22.875806,26.767233,27.448077,19.635821,19.894821
24810,2023-10-31 21:00:00,29.730000,0.0,27.209630,14.012000,18.277500,18.075000,21.498163,20.048571,21.667453,...,19.601459,22.618684,18.950682,0.00,0.000000,24.917614,28.554130,30.371875,20.090000,19.403636
24811,2023-10-31 22:00:00,31.780000,0.0,32.099286,27.931111,9.445833,23.772500,20.993243,20.301818,23.467257,...,19.009235,22.166389,19.928830,0.00,0.000000,19.226604,26.789421,30.670833,18.930952,19.634500


In [47]:
## dataframes for each aggregation

# Schools
zones_with_school = pd.read_csv('zones_with_school.csv')
zones_with_school_pri = pd.read_csv('zones_with_school_pri.csv')
zones_with_school_sec = pd.read_csv('zones_with_school_sec.csv')
zones_with_school_uni = pd.read_csv('zones_with_school_uni.csv')

# Food
zones_with_food = pd.read_csv('zones_with_food.csv')
zones_with_food_bar = pd.read_csv('zones_with_food_bar.csv')
zones_with_food_morning = pd.read_csv('zones_with_food_morning.csv')
zones_with_food_restaurant = pd.read_csv('zones_with_food_restaurant.csv')

# Attractions
zones_with_attractions = pd.read_csv('zones_with_attractions.csv')

# Religious
zones_with_religious = pd.read_csv('zones_with_religious.csv')

# School 15min

In [48]:
# dd df for zones_with_school
selected_columns = zones_with_school['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_school = combined_df[selected_columns]
dd_df_school

,time_slot,13,24,41,48,75,100,113,114,116,...,166,202,211,231,232,236,238,239,244,261
0,2021-01-01 00:00:00,1.0,2.0,2.0,20.0,4.0,10.0,6.0,2.0,0.0,...,3.0,0.0,1.0,1.0,0.0,5.0,3.0,5.0,0.0,1.0
1,2021-01-01 00:15:00,1.0,2.0,2.0,20.0,5.0,7.0,1.0,2.0,0.0,...,1.0,0.0,1.0,1.0,0.0,9.0,11.0,5.0,0.0,1.0
2,2021-01-01 00:30:00,0.0,0.0,1.0,21.0,6.0,9.0,5.0,1.0,1.0,...,4.0,0.0,2.0,3.0,1.0,12.0,9.0,6.0,2.0,0.0
3,2021-01-01 00:45:00,1.0,3.0,0.0,20.0,7.0,9.0,5.0,3.0,1.0,...,1.0,0.0,1.0,7.0,1.0,9.0,16.0,12.0,0.0,0.0
4,2021-01-01 01:00:00,0.0,1.0,9.0,18.0,3.0,11.0,2.0,2.0,1.0,...,3.0,0.0,4.0,4.0,3.0,4.0,7.0,15.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,5.0,1.0,3.0,52.0,3.0,15.0,36.0,19.0,2.0,...,4.0,0.0,19.0,16.0,1.0,18.0,6.0,20.0,0.0,4.0
99248,2023-10-31 23:00:00,3.0,1.0,1.0,34.0,0.0,13.0,35.0,12.0,0.0,...,1.0,0.0,21.0,19.0,2.0,9.0,6.0,9.0,0.0,3.0
99249,2023-10-31 23:15:00,1.0,1.0,0.0,39.0,2.0,4.0,47.0,22.0,0.0,...,2.0,0.0,24.0,9.0,2.0,8.0,7.0,14.0,1.0,1.0
99250,2023-10-31 23:30:00,3.0,0.0,1.0,44.0,0.0,10.0,44.0,36.0,2.0,...,0.0,0.0,11.0,12.0,0.0,9.0,7.0,15.0,1.0,1.0


In [49]:
dd_df_school.to_csv('dd_df_school.csv', index=False, header=True)

In [50]:
# total fair amount df for zones_with_school
total_fair_df_school = combined_total_fair_df[selected_columns]
total_fair_df_school

,time_slot,13,24,41,48,75,100,113,114,116,...,166,202,211,231,232,236,238,239,244,261
0,2021-01-01 00:00:00,28.300000,16.87,19.205000,15.230000,16.090000,14.464000,21.086667,24.925000,0.00,...,11.616667,0.0,11.300000,10.120000,0.000000,18.064000,13.790000,12.522000,0.00,25.300000
1,2021-01-01 00:15:00,23.800000,14.60,20.865000,17.816000,14.590000,12.177143,22.880000,15.955000,0.00,...,3.800000,0.0,15.380000,9.360000,0.000000,11.132222,14.593636,12.618000,0.00,23.160000
2,2021-01-01 00:30:00,0.000000,0.00,14.650000,15.241429,10.580000,18.425556,18.234000,9.300000,24.80,...,10.275000,0.0,11.000000,13.530000,29.800000,13.081667,14.180000,14.355000,18.17,0.000000
3,2021-01-01 00:45:00,21.800000,19.20,0.000000,19.137500,28.478571,24.711111,26.010000,20.600000,11.75,...,11.760000,0.0,20.160000,21.941429,17.750000,12.782222,17.263125,15.713333,0.00,0.000000
4,2021-01-01 01:00:00,0.000000,14.80,24.674444,13.168889,26.533333,26.825455,11.075000,18.290000,6.80,...,27.853333,0.0,17.802500,31.635000,26.163333,13.615000,15.618571,13.231333,0.00,39.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,43.092000,21.70,11.453333,23.352308,15.920000,26.085333,29.217222,22.039474,36.82,...,25.317500,0.0,25.754737,27.527500,42.200000,21.780000,22.686667,16.312500,0.00,35.407500
99248,2023-10-31 23:00:00,32.426667,23.70,35.050000,16.966176,0.000000,27.268462,28.443714,32.678333,0.00,...,23.040000,0.0,24.074286,30.584737,20.760000,29.883333,27.188333,22.350000,0.00,32.003333
99249,2023-10-31 23:15:00,31.440000,13.60,0.000000,22.244615,19.380000,17.650000,26.004468,21.295909,0.00,...,12.210000,0.0,26.732500,30.415556,33.960000,21.703750,22.872857,21.289286,22.55,18.840000
99250,2023-10-31 23:30:00,19.006667,0.00,35.640000,23.023409,0.000000,23.213000,25.276136,29.422500,15.42,...,0.000000,0.0,21.369091,29.155833,0.000000,19.304444,17.165714,19.174667,43.20,46.560000


In [51]:
total_fair_df_school.to_csv('total_fair_df_school.csv', index=False, header=True)

# Pri School

In [53]:
# dd df for zones_with_school_pri
selected_columns = zones_with_school_pri['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_school_pri = combined_df[selected_columns]
dd_df_school_pri

,time_slot,137,232
0,2021-01-01 00:00:00,6.0,0.0
1,2021-01-01 00:15:00,7.0,0.0
2,2021-01-01 00:30:00,5.0,1.0
3,2021-01-01 00:45:00,5.0,1.0
4,2021-01-01 01:00:00,5.0,3.0
...,...,...,...
99247,2023-10-31 22:45:00,12.0,1.0
99248,2023-10-31 23:00:00,11.0,2.0
99249,2023-10-31 23:15:00,13.0,2.0
99250,2023-10-31 23:30:00,8.0,0.0


In [54]:
dd_df_school_pri.to_csv('dd_df_school_pri.csv', index=False, header=True)

In [55]:
# total fair amount df for zones_with_school
total_fair_df_school_pri = combined_total_fair_df[selected_columns]
total_fair_df_school_pri

,time_slot,137,232
0,2021-01-01 00:00:00,21.310000,0.000000
1,2021-01-01 00:15:00,15.020000,0.000000
2,2021-01-01 00:30:00,16.750000,29.800000
3,2021-01-01 00:45:00,21.470000,17.750000
4,2021-01-01 01:00:00,18.912000,26.163333
...,...,...,...
99247,2023-10-31 22:45:00,22.170833,42.200000
99248,2023-10-31 23:00:00,19.781818,20.760000
99249,2023-10-31 23:15:00,24.097692,33.960000
99250,2023-10-31 23:30:00,13.111250,0.000000


In [56]:
total_fair_df_school_pri.to_csv('total_fair_df_school_pri.csv', index=False, header=True)

# Sec School

In [57]:
# dd df for zones_with_school
selected_columns = zones_with_school_sec['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_school_sec = combined_df[selected_columns]
dd_df_school_sec

,time_slot,75,137,236,261
0,2021-01-01 00:00:00,4.0,6.0,5.0,1.0
1,2021-01-01 00:15:00,5.0,7.0,9.0,1.0
2,2021-01-01 00:30:00,6.0,5.0,12.0,0.0
3,2021-01-01 00:45:00,7.0,5.0,9.0,0.0
4,2021-01-01 01:00:00,3.0,5.0,4.0,1.0
...,...,...,...,...,...
99247,2023-10-31 22:45:00,3.0,12.0,18.0,4.0
99248,2023-10-31 23:00:00,0.0,11.0,9.0,3.0
99249,2023-10-31 23:15:00,2.0,13.0,8.0,1.0
99250,2023-10-31 23:30:00,0.0,8.0,9.0,1.0


In [58]:
dd_df_school_sec.to_csv('dd_df_school_sec.csv', index=False, header=True)

In [59]:
# total fair amount df for zones_with_school
total_fair_df_school_sec = combined_total_fair_df[selected_columns]
total_fair_df_school_sec

,time_slot,75,137,236,261
0,2021-01-01 00:00:00,16.090000,21.310000,18.064000,25.300000
1,2021-01-01 00:15:00,14.590000,15.020000,11.132222,23.160000
2,2021-01-01 00:30:00,10.580000,16.750000,13.081667,0.000000
3,2021-01-01 00:45:00,28.478571,21.470000,12.782222,0.000000
4,2021-01-01 01:00:00,26.533333,18.912000,13.615000,39.300000
...,...,...,...,...,...
99247,2023-10-31 22:45:00,15.920000,22.170833,21.780000,35.407500
99248,2023-10-31 23:00:00,0.000000,19.781818,29.883333,32.003333
99249,2023-10-31 23:15:00,19.380000,24.097692,21.703750,18.840000
99250,2023-10-31 23:30:00,0.000000,13.111250,19.304444,46.560000


In [60]:
total_fair_df_school_sec.to_csv('total_fair_df_school_sec.csv', index=False, header=True)

# Uni

In [61]:
# dd df for zones_with_school
selected_columns = zones_with_school_uni['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_school_uni = combined_df[selected_columns]
dd_df_school_uni

,time_slot,48,113,114,142,164,166,238,244
0,2021-01-01 00:00:00,20.0,6.0,2.0,2.0,6.0,3.0,3.0,0.0
1,2021-01-01 00:15:00,20.0,1.0,2.0,8.0,7.0,1.0,11.0,0.0
2,2021-01-01 00:30:00,21.0,5.0,1.0,17.0,5.0,4.0,9.0,2.0
3,2021-01-01 00:45:00,20.0,5.0,3.0,10.0,7.0,1.0,16.0,0.0
4,2021-01-01 01:00:00,18.0,2.0,2.0,18.0,5.0,3.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,52.0,36.0,19.0,33.0,14.0,4.0,6.0,0.0
99248,2023-10-31 23:00:00,34.0,35.0,12.0,38.0,22.0,1.0,6.0,0.0
99249,2023-10-31 23:15:00,39.0,47.0,22.0,118.0,14.0,2.0,7.0,1.0
99250,2023-10-31 23:30:00,44.0,44.0,36.0,43.0,19.0,0.0,7.0,1.0


In [62]:
dd_df_school_uni.to_csv('dd_df_school_uni.csv', index=False, header=True)

In [63]:
# total fair amount df for zones_with_school
total_fair_df_school_uni = combined_total_fair_df[selected_columns]
total_fair_df_school_uni

,time_slot,48,113,114,142,164,166,238,244
0,2021-01-01 00:00:00,15.230000,21.086667,24.925000,17.900000,17.025000,11.616667,13.790000,0.00
1,2021-01-01 00:15:00,17.816000,22.880000,15.955000,15.125000,24.017143,3.800000,14.593636,0.00
2,2021-01-01 00:30:00,15.241429,18.234000,9.300000,19.153529,16.338000,10.275000,14.180000,18.17
3,2021-01-01 00:45:00,19.137500,26.010000,20.600000,14.283000,16.411429,11.760000,17.263125,0.00
4,2021-01-01 01:00:00,13.168889,11.075000,18.290000,18.282222,19.832000,27.853333,15.618571,0.00
...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,23.352308,29.217222,22.039474,25.409697,20.539286,25.317500,22.686667,0.00
99248,2023-10-31 23:00:00,16.966176,28.443714,32.678333,23.725789,20.097727,23.040000,27.188333,0.00
99249,2023-10-31 23:15:00,22.244615,26.004468,21.295909,23.156356,24.387143,12.210000,22.872857,22.55
99250,2023-10-31 23:30:00,23.023409,25.276136,29.422500,33.357907,21.045263,0.000000,17.165714,43.20


In [64]:
total_fair_df_school_uni.to_csv('total_fair_df_school_uni.csv', index=False, header=True)

# Food

In [69]:
# dd df for zones_with_school
selected_columns = zones_with_food['PULocationID'].values
selected_columns

array([  4,  13,  41,  42,  45,  48,  50,  79,  87,  88,  90, 100, 103,
       113, 114, 116, 125, 127, 137, 140, 141, 142, 144, 148, 151, 158,
       161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 230, 231, 233,
       234, 238, 239, 243, 244, 249, 261, 262, 263])

In [72]:
# exclude zone 103 as it is not in the time series for 2021-2023
selected_columns = ['time_slot'] + [str(x) for x in selected_columns if x != 103]
dd_df_food = combined_df[selected_columns]
dd_df_food

,time_slot,4,13,41,42,45,48,50,79,87,...,233,234,238,239,243,244,249,261,262,263
0,2021-01-01 00:00:00,1.0,1.0,2.0,0.0,1.0,20.0,1.0,3.0,0.0,...,2.0,5.0,3.0,5.0,0.0,0.0,5.0,1.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,1.0,2.0,2.0,2.0,20.0,1.0,7.0,1.0,...,2.0,9.0,11.0,5.0,0.0,0.0,1.0,1.0,2.0,16.0
2,2021-01-01 00:30:00,1.0,0.0,1.0,0.0,1.0,21.0,3.0,3.0,0.0,...,6.0,4.0,9.0,6.0,0.0,2.0,6.0,0.0,4.0,14.0
3,2021-01-01 00:45:00,0.0,1.0,0.0,0.0,0.0,20.0,2.0,8.0,5.0,...,3.0,4.0,16.0,12.0,0.0,0.0,8.0,0.0,8.0,17.0
4,2021-01-01 01:00:00,0.0,0.0,9.0,5.0,0.0,18.0,1.0,7.0,2.0,...,2.0,6.0,7.0,15.0,0.0,0.0,12.0,1.0,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,5.0,3.0,0.0,1.0,52.0,8.0,58.0,4.0,...,10.0,55.0,6.0,20.0,0.0,0.0,60.0,4.0,3.0,15.0
99248,2023-10-31 23:00:00,3.0,3.0,1.0,0.0,4.0,34.0,5.0,42.0,2.0,...,5.0,43.0,6.0,9.0,0.0,0.0,65.0,3.0,6.0,15.0
99249,2023-10-31 23:15:00,0.0,1.0,0.0,2.0,1.0,39.0,5.0,51.0,7.0,...,4.0,36.0,7.0,14.0,0.0,1.0,68.0,1.0,3.0,15.0
99250,2023-10-31 23:30:00,1.0,3.0,1.0,1.0,4.0,44.0,3.0,42.0,3.0,...,5.0,33.0,7.0,15.0,0.0,1.0,76.0,1.0,2.0,9.0


In [73]:
dd_df_food.to_csv('dd_df_food.csv', index=False, header=True)

In [74]:
# total fair amount df for zones_with_school
total_fair_df_food = combined_total_fair_df[selected_columns]
total_fair_df_food

,time_slot,4,13,41,42,45,48,50,79,87,...,233,234,238,239,243,244,249,261,262,263
0,2021-01-01 00:00:00,22.800000,28.300000,19.205000,0.000,30.36,15.230000,12.800000,29.770000,0.000000,...,16.550000,38.494000,13.790000,12.522000,0.0,0.00,22.582000,25.300000,8.300000,7.071429
1,2021-01-01 00:15:00,27.800000,23.800000,20.865000,31.050,0.00,17.816000,10.300000,20.871429,15.950000,...,12.130000,15.374444,14.593636,12.618000,0.0,0.00,12.800000,23.160000,18.975000,10.588750
2,2021-01-01 00:30:00,13.800000,0.000000,14.650000,0.000,63.42,15.241429,13.650000,30.300000,0.000000,...,16.425000,14.075000,14.180000,14.355000,0.0,18.17,16.703333,0.000000,13.850000,14.023571
3,2021-01-01 00:45:00,0.000000,21.800000,0.000000,0.000,0.00,19.137500,26.700000,12.756250,25.570000,...,16.633333,25.125000,17.263125,15.713333,0.0,0.00,23.570000,0.000000,14.260000,14.249412
4,2021-01-01 01:00:00,0.000000,0.000000,24.674444,18.000,0.00,13.168889,32.800000,17.291429,20.420000,...,20.155000,12.290000,15.618571,13.231333,0.0,0.00,22.605833,39.300000,9.584000,17.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,43.092000,11.453333,0.000,14.64,23.352308,26.767500,24.737586,23.765000,...,25.366000,23.601818,22.686667,16.312500,0.0,0.00,26.023167,35.407500,20.640000,13.883333
99248,2023-10-31 23:00:00,20.256667,32.426667,35.050000,0.000,35.65,16.966176,24.480000,27.496667,18.420000,...,32.804000,23.971395,27.188333,22.350000,0.0,0.00,27.037846,32.003333,15.461667,20.020667
99249,2023-10-31 23:15:00,0.000000,31.440000,0.000000,31.085,18.70,22.244615,21.038000,23.124706,29.142857,...,22.727500,25.498611,22.872857,21.289286,0.0,22.55,26.830588,18.840000,12.190000,17.386000
99250,2023-10-31 23:30:00,66.880000,19.006667,35.640000,11.100,22.12,23.023409,18.133333,20.611190,21.033333,...,24.872000,26.507879,17.165714,19.174667,0.0,43.20,25.789605,46.560000,14.585000,20.304444


In [75]:
total_fair_df_food.to_csv('total_fair_df_food.csv', index=False, header=True)

# Food Bar

In [76]:
# dd df for zones_with_school
selected_columns = zones_with_food_bar['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_food_bar = combined_df[selected_columns]
dd_df_food_bar

,time_slot,4,13,45,48,50,79,88,100,125,...,186,230,231,233,234,239,243,249,261,263
0,2021-01-01 00:00:00,1.0,1.0,1.0,20.0,1.0,3.0,1.0,10.0,0.0,...,2.0,15.0,1.0,2.0,5.0,5.0,0.0,5.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,1.0,2.0,20.0,1.0,7.0,0.0,7.0,1.0,...,3.0,16.0,1.0,2.0,9.0,5.0,0.0,1.0,1.0,16.0
2,2021-01-01 00:30:00,1.0,0.0,1.0,21.0,3.0,3.0,0.0,9.0,2.0,...,3.0,15.0,3.0,6.0,4.0,6.0,0.0,6.0,0.0,14.0
3,2021-01-01 00:45:00,0.0,1.0,0.0,20.0,2.0,8.0,0.0,9.0,1.0,...,11.0,13.0,7.0,3.0,4.0,12.0,0.0,8.0,0.0,17.0
4,2021-01-01 01:00:00,0.0,0.0,0.0,18.0,1.0,7.0,0.0,11.0,2.0,...,10.0,18.0,4.0,2.0,6.0,15.0,0.0,12.0,1.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,5.0,1.0,52.0,8.0,58.0,1.0,15.0,11.0,...,36.0,60.0,16.0,10.0,55.0,20.0,0.0,60.0,4.0,15.0
99248,2023-10-31 23:00:00,3.0,3.0,4.0,34.0,5.0,42.0,2.0,13.0,10.0,...,22.0,41.0,19.0,5.0,43.0,9.0,0.0,65.0,3.0,15.0
99249,2023-10-31 23:15:00,0.0,1.0,1.0,39.0,5.0,51.0,4.0,4.0,3.0,...,17.0,39.0,9.0,4.0,36.0,14.0,0.0,68.0,1.0,15.0
99250,2023-10-31 23:30:00,1.0,3.0,4.0,44.0,3.0,42.0,3.0,10.0,5.0,...,17.0,27.0,12.0,5.0,33.0,15.0,0.0,76.0,1.0,9.0


In [77]:
dd_df_food_bar.to_csv('dd_df_food_bar.csv', index=False, header=True)

In [78]:
# total fair amount df for zones_with_school
total_fair_df_food_bar = combined_total_fair_df[selected_columns]
total_fair_df_food_bar

,time_slot,4,13,45,48,50,79,88,100,125,...,186,230,231,233,234,239,243,249,261,263
0,2021-01-01 00:00:00,22.800000,28.300000,30.36,15.230000,12.800000,29.770000,27.950000,14.464000,0.000000,...,21.050000,16.750667,10.120000,16.550000,38.494000,12.522000,0.0,22.582000,25.300000,7.071429
1,2021-01-01 00:15:00,27.800000,23.800000,0.00,17.816000,10.300000,20.871429,0.000000,12.177143,12.360000,...,11.633333,27.193125,9.360000,12.130000,15.374444,12.618000,0.0,12.800000,23.160000,10.588750
2,2021-01-01 00:30:00,13.800000,0.000000,63.42,15.241429,13.650000,30.300000,0.000000,18.425556,21.625000,...,9.486667,14.406000,13.530000,16.425000,14.075000,14.355000,0.0,16.703333,0.000000,14.023571
3,2021-01-01 00:45:00,0.000000,21.800000,0.00,19.137500,26.700000,12.756250,0.000000,24.711111,10.560000,...,21.031818,22.180000,21.941429,16.633333,25.125000,15.713333,0.0,23.570000,0.000000,14.249412
4,2021-01-01 01:00:00,0.000000,0.000000,0.00,13.168889,32.800000,17.291429,0.000000,26.825455,26.855000,...,24.388000,21.717222,31.635000,20.155000,12.290000,13.231333,0.0,22.605833,39.300000,17.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,43.092000,14.64,23.352308,26.767500,24.737586,12.960000,26.085333,23.098182,...,24.779444,25.934333,27.527500,25.366000,23.601818,16.312500,0.0,26.023167,35.407500,13.883333
99248,2023-10-31 23:00:00,20.256667,32.426667,35.65,16.966176,24.480000,27.496667,53.300000,27.268462,27.149000,...,28.292273,21.359756,30.584737,32.804000,23.971395,22.350000,0.0,27.037846,32.003333,20.020667
99249,2023-10-31 23:15:00,0.000000,31.440000,18.70,22.244615,21.038000,23.124706,34.365000,17.650000,27.490000,...,33.765882,25.594615,30.415556,22.727500,25.498611,21.289286,0.0,26.830588,18.840000,17.386000
99250,2023-10-31 23:30:00,66.880000,19.006667,22.12,23.023409,18.133333,20.611190,29.426667,23.213000,18.612000,...,26.274706,22.333704,29.155833,24.872000,26.507879,19.174667,0.0,25.789605,46.560000,20.304444


In [79]:
total_fair_df_food_bar.to_csv('total_fair_df_food_bar.csv', index=False, header=True)

# Food Morning

In [80]:
# dd df for zones_with_school
selected_columns = zones_with_food_morning['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_food_morning = combined_df[selected_columns]
dd_df_food_morning

,time_slot,13,48,50,79,87,114,140,141,142,...,166,186,194,231,233,238,239,249,262,263
0,2021-01-01 00:00:00,1.0,20.0,1.0,3.0,0.0,2.0,3.0,3.0,2.0,...,3.0,2.0,0.0,1.0,2.0,3.0,5.0,5.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,20.0,1.0,7.0,1.0,2.0,6.0,8.0,8.0,...,1.0,3.0,0.0,1.0,2.0,11.0,5.0,1.0,2.0,16.0
2,2021-01-01 00:30:00,0.0,21.0,3.0,3.0,0.0,1.0,5.0,12.0,17.0,...,4.0,3.0,0.0,3.0,6.0,9.0,6.0,6.0,4.0,14.0
3,2021-01-01 00:45:00,1.0,20.0,2.0,8.0,5.0,3.0,5.0,16.0,10.0,...,1.0,11.0,0.0,7.0,3.0,16.0,12.0,8.0,8.0,17.0
4,2021-01-01 01:00:00,0.0,18.0,1.0,7.0,2.0,2.0,3.0,11.0,18.0,...,3.0,10.0,0.0,4.0,2.0,7.0,15.0,12.0,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,5.0,52.0,8.0,58.0,4.0,19.0,11.0,17.0,33.0,...,4.0,36.0,0.0,16.0,10.0,6.0,20.0,60.0,3.0,15.0
99248,2023-10-31 23:00:00,3.0,34.0,5.0,42.0,2.0,12.0,14.0,14.0,38.0,...,1.0,22.0,0.0,19.0,5.0,6.0,9.0,65.0,6.0,15.0
99249,2023-10-31 23:15:00,1.0,39.0,5.0,51.0,7.0,22.0,10.0,10.0,118.0,...,2.0,17.0,0.0,9.0,4.0,7.0,14.0,68.0,3.0,15.0
99250,2023-10-31 23:30:00,3.0,44.0,3.0,42.0,3.0,36.0,7.0,10.0,43.0,...,0.0,17.0,0.0,12.0,5.0,7.0,15.0,76.0,2.0,9.0


In [81]:
dd_df_food_morning.to_csv('dd_df_food_bar.csv', index=False, header=True)

In [82]:
# total fair amount df for zones_with_school
total_fair_df_food_morning = combined_total_fair_df[selected_columns]
total_fair_df_food_morning

,time_slot,13,48,50,79,87,114,140,141,142,...,166,186,194,231,233,238,239,249,262,263
0,2021-01-01 00:00:00,28.300000,15.230000,12.800000,29.770000,0.000000,24.925000,14.453333,13.300000,17.900000,...,11.616667,21.050000,0.0,10.120000,16.550000,13.790000,12.522000,22.582000,8.300000,7.071429
1,2021-01-01 00:15:00,23.800000,17.816000,10.300000,20.871429,15.950000,15.955000,16.840000,12.167500,15.125000,...,3.800000,11.633333,0.0,9.360000,12.130000,14.593636,12.618000,12.800000,18.975000,10.588750
2,2021-01-01 00:30:00,0.000000,15.241429,13.650000,30.300000,0.000000,9.300000,13.308000,15.065833,19.153529,...,10.275000,9.486667,0.0,13.530000,16.425000,14.180000,14.355000,16.703333,13.850000,14.023571
3,2021-01-01 00:45:00,21.800000,19.137500,26.700000,12.756250,25.570000,20.600000,17.304000,15.103750,14.283000,...,11.760000,21.031818,0.0,21.941429,16.633333,17.263125,15.713333,23.570000,14.260000,14.249412
4,2021-01-01 01:00:00,0.000000,13.168889,32.800000,17.291429,20.420000,18.290000,17.733333,12.451818,18.282222,...,27.853333,24.388000,0.0,31.635000,20.155000,15.618571,13.231333,22.605833,9.584000,17.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,43.092000,23.352308,26.767500,24.737586,23.765000,22.039474,24.420000,16.827647,25.409697,...,25.317500,24.779444,0.0,27.527500,25.366000,22.686667,16.312500,26.023167,20.640000,13.883333
99248,2023-10-31 23:00:00,32.426667,16.966176,24.480000,27.496667,18.420000,32.678333,19.025000,23.120000,23.725789,...,23.040000,28.292273,0.0,30.584737,32.804000,27.188333,22.350000,27.037846,15.461667,20.020667
99249,2023-10-31 23:15:00,31.440000,22.244615,21.038000,23.124706,29.142857,21.295909,21.360000,21.230000,23.156356,...,12.210000,33.765882,0.0,30.415556,22.727500,22.872857,21.289286,26.830588,12.190000,17.386000
99250,2023-10-31 23:30:00,19.006667,23.023409,18.133333,20.611190,21.033333,29.422500,18.872857,25.223000,33.357907,...,0.000000,26.274706,0.0,29.155833,24.872000,17.165714,19.174667,25.789605,14.585000,20.304444


In [83]:
total_fair_df_food_morning.to_csv('total_fair_df_food_morning.csv', index=False, header=True)

# Food Restaurant 

In [104]:
# dd df for zones_with_school
selected_columns = zones_with_food_restaurant['PULocationID'].values
selected_columns

array([  4,  13,  41,  42,  45,  48,  50,  79,  88,  90, 100, 103, 113,
       116, 125, 127, 137, 140, 141, 142, 144, 148, 151, 158, 161, 162,
       163, 164, 170, 186, 202, 209, 230, 231, 233, 234, 238, 239, 243,
       244, 249, 261, 262, 263])

In [106]:
# exclude zone 103 as it is not in the time series for 2021-2023
selected_columns = ['time_slot'] + [str(x) for x in selected_columns if x != 103]
dd_df_food_restaurant = combined_df[selected_columns]
dd_df_food_restaurant

,time_slot,4,13,41,42,45,48,50,79,88,...,233,234,238,239,243,244,249,261,262,263
0,2021-01-01 00:00:00,1.0,1.0,2.0,0.0,1.0,20.0,1.0,3.0,1.0,...,2.0,5.0,3.0,5.0,0.0,0.0,5.0,1.0,1.0,7.0
1,2021-01-01 00:15:00,1.0,1.0,2.0,2.0,2.0,20.0,1.0,7.0,0.0,...,2.0,9.0,11.0,5.0,0.0,0.0,1.0,1.0,2.0,16.0
2,2021-01-01 00:30:00,1.0,0.0,1.0,0.0,1.0,21.0,3.0,3.0,0.0,...,6.0,4.0,9.0,6.0,0.0,2.0,6.0,0.0,4.0,14.0
3,2021-01-01 00:45:00,0.0,1.0,0.0,0.0,0.0,20.0,2.0,8.0,0.0,...,3.0,4.0,16.0,12.0,0.0,0.0,8.0,0.0,8.0,17.0
4,2021-01-01 01:00:00,0.0,0.0,9.0,5.0,0.0,18.0,1.0,7.0,0.0,...,2.0,6.0,7.0,15.0,0.0,0.0,12.0,1.0,5.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,5.0,3.0,0.0,1.0,52.0,8.0,58.0,1.0,...,10.0,55.0,6.0,20.0,0.0,0.0,60.0,4.0,3.0,15.0
99248,2023-10-31 23:00:00,3.0,3.0,1.0,0.0,4.0,34.0,5.0,42.0,2.0,...,5.0,43.0,6.0,9.0,0.0,0.0,65.0,3.0,6.0,15.0
99249,2023-10-31 23:15:00,0.0,1.0,0.0,2.0,1.0,39.0,5.0,51.0,4.0,...,4.0,36.0,7.0,14.0,0.0,1.0,68.0,1.0,3.0,15.0
99250,2023-10-31 23:30:00,1.0,3.0,1.0,1.0,4.0,44.0,3.0,42.0,3.0,...,5.0,33.0,7.0,15.0,0.0,1.0,76.0,1.0,2.0,9.0


In [107]:
dd_df_food_restaurant.to_csv('dd_df_food_restaurant.csv', index=False, header=True)

In [108]:
# total fair amount df for zones_with_school
total_fair_df_food_restaurant = combined_total_fair_df[selected_columns]
total_fair_df_food_restaurant

,time_slot,4,13,41,42,45,48,50,79,88,...,233,234,238,239,243,244,249,261,262,263
0,2021-01-01 00:00:00,22.800000,28.300000,19.205000,0.000,30.36,15.230000,12.800000,29.770000,27.950000,...,16.550000,38.494000,13.790000,12.522000,0.0,0.00,22.582000,25.300000,8.300000,7.071429
1,2021-01-01 00:15:00,27.800000,23.800000,20.865000,31.050,0.00,17.816000,10.300000,20.871429,0.000000,...,12.130000,15.374444,14.593636,12.618000,0.0,0.00,12.800000,23.160000,18.975000,10.588750
2,2021-01-01 00:30:00,13.800000,0.000000,14.650000,0.000,63.42,15.241429,13.650000,30.300000,0.000000,...,16.425000,14.075000,14.180000,14.355000,0.0,18.17,16.703333,0.000000,13.850000,14.023571
3,2021-01-01 00:45:00,0.000000,21.800000,0.000000,0.000,0.00,19.137500,26.700000,12.756250,0.000000,...,16.633333,25.125000,17.263125,15.713333,0.0,0.00,23.570000,0.000000,14.260000,14.249412
4,2021-01-01 01:00:00,0.000000,0.000000,24.674444,18.000,0.00,13.168889,32.800000,17.291429,0.000000,...,20.155000,12.290000,15.618571,13.231333,0.0,0.00,22.605833,39.300000,9.584000,17.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,43.092000,11.453333,0.000,14.64,23.352308,26.767500,24.737586,12.960000,...,25.366000,23.601818,22.686667,16.312500,0.0,0.00,26.023167,35.407500,20.640000,13.883333
99248,2023-10-31 23:00:00,20.256667,32.426667,35.050000,0.000,35.65,16.966176,24.480000,27.496667,53.300000,...,32.804000,23.971395,27.188333,22.350000,0.0,0.00,27.037846,32.003333,15.461667,20.020667
99249,2023-10-31 23:15:00,0.000000,31.440000,0.000000,31.085,18.70,22.244615,21.038000,23.124706,34.365000,...,22.727500,25.498611,22.872857,21.289286,0.0,22.55,26.830588,18.840000,12.190000,17.386000
99250,2023-10-31 23:30:00,66.880000,19.006667,35.640000,11.100,22.12,23.023409,18.133333,20.611190,29.426667,...,24.872000,26.507879,17.165714,19.174667,0.0,43.20,25.789605,46.560000,14.585000,20.304444


In [109]:
total_fair_df_food_restaurant.to_csv('total_fair_df_food_restaurant.csv', index=False, header=True)

# Attractions

In [110]:
# dd df for zones_with_school
selected_columns = zones_with_attractions['PULocationID'].values
selected_columns

array([  4,  12,  13,  45,  48,  50,  75,  79,  87, 103, 113, 114, 120,
       137, 142, 148, 202, 209, 211, 230, 232, 234])

In [111]:
# exclude zone 103 as it is not in the time series for 2021-2023
selected_columns = ['time_slot'] + [str(x) for x in selected_columns if x != 103]
dd_df_attractions = combined_df[selected_columns]
dd_df_attractions

,time_slot,4,12,13,45,48,50,75,79,87,...,120,137,142,148,202,209,211,230,232,234
0,2021-01-01 00:00:00,1.0,0.0,1.0,1.0,20.0,1.0,4.0,3.0,0.0,...,0.0,6.0,2.0,2.0,0.0,1.0,1.0,15.0,0.0,5.0
1,2021-01-01 00:15:00,1.0,0.0,1.0,2.0,20.0,1.0,5.0,7.0,1.0,...,0.0,7.0,8.0,1.0,0.0,0.0,1.0,16.0,0.0,9.0
2,2021-01-01 00:30:00,1.0,0.0,0.0,1.0,21.0,3.0,6.0,3.0,0.0,...,0.0,5.0,17.0,5.0,0.0,1.0,2.0,15.0,1.0,4.0
3,2021-01-01 00:45:00,0.0,0.0,1.0,0.0,20.0,2.0,7.0,8.0,5.0,...,0.0,5.0,10.0,1.0,0.0,1.0,1.0,13.0,1.0,4.0
4,2021-01-01 01:00:00,0.0,0.0,0.0,0.0,18.0,1.0,3.0,7.0,2.0,...,0.0,5.0,18.0,5.0,0.0,0.0,4.0,18.0,3.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,0.0,5.0,1.0,52.0,8.0,3.0,58.0,4.0,...,0.0,12.0,33.0,14.0,0.0,1.0,19.0,60.0,1.0,55.0
99248,2023-10-31 23:00:00,3.0,0.0,3.0,4.0,34.0,5.0,0.0,42.0,2.0,...,0.0,11.0,38.0,11.0,0.0,2.0,21.0,41.0,2.0,43.0
99249,2023-10-31 23:15:00,0.0,0.0,1.0,1.0,39.0,5.0,2.0,51.0,7.0,...,0.0,13.0,118.0,23.0,0.0,1.0,24.0,39.0,2.0,36.0
99250,2023-10-31 23:30:00,1.0,1.0,3.0,4.0,44.0,3.0,0.0,42.0,3.0,...,0.0,8.0,43.0,24.0,0.0,1.0,11.0,27.0,0.0,33.0


In [112]:
dd_df_attractions.to_csv('dd_df_attractions.csv', index=False, header=True)

In [113]:
# total fair amount df for zones_with_school
total_fair_df_attractions = combined_total_fair_df[selected_columns]
total_fair_df_attractions

,time_slot,4,12,13,45,48,50,75,79,87,...,120,137,142,148,202,209,211,230,232,234
0,2021-01-01 00:00:00,22.800000,0.0,28.300000,30.36,15.230000,12.800000,16.090000,29.770000,0.000000,...,0.0,21.310000,17.900000,17.380000,0.0,23.75,11.300000,16.750667,0.000000,38.494000
1,2021-01-01 00:15:00,27.800000,0.0,23.800000,0.00,17.816000,10.300000,14.590000,20.871429,15.950000,...,0.0,15.020000,15.125000,8.300000,0.0,0.00,15.380000,27.193125,0.000000,15.374444
2,2021-01-01 00:30:00,13.800000,0.0,0.000000,63.42,15.241429,13.650000,10.580000,30.300000,0.000000,...,0.0,16.750000,19.153529,17.242000,0.0,36.24,11.000000,14.406000,29.800000,14.075000
3,2021-01-01 00:45:00,0.000000,0.0,21.800000,0.00,19.137500,26.700000,28.478571,12.756250,25.570000,...,0.0,21.470000,14.283000,22.800000,0.0,24.96,20.160000,22.180000,17.750000,25.125000
4,2021-01-01 01:00:00,0.000000,0.0,0.000000,0.00,13.168889,32.800000,26.533333,17.291429,20.420000,...,0.0,18.912000,18.282222,21.584000,0.0,0.00,17.802500,21.717222,26.163333,12.290000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,0.0,43.092000,14.64,23.352308,26.767500,15.920000,24.737586,23.765000,...,0.0,22.170833,25.409697,19.653571,0.0,26.99,25.754737,25.934333,42.200000,23.601818
99248,2023-10-31 23:00:00,20.256667,0.0,32.426667,35.65,16.966176,24.480000,0.000000,27.496667,18.420000,...,0.0,19.781818,23.725789,29.054545,0.0,22.38,24.074286,21.359756,20.760000,23.971395
99249,2023-10-31 23:15:00,0.000000,0.0,31.440000,18.70,22.244615,21.038000,19.380000,23.124706,29.142857,...,0.0,24.097692,23.156356,27.392609,0.0,10.80,26.732500,25.594615,33.960000,25.498611
99250,2023-10-31 23:30:00,66.880000,26.4,19.006667,22.12,23.023409,18.133333,0.000000,20.611190,21.033333,...,0.0,13.111250,33.357907,21.800417,0.0,57.45,21.369091,22.333704,0.000000,26.507879


In [114]:
total_fair_df_attractions.to_csv('total_fair_df_attractions.csv', index=False, header=True)

# Religious

In [115]:
# dd df for zones_with_school
selected_columns = zones_with_religious['PULocationID'].values
selected_columns = ['time_slot'] + [str(x) for x in selected_columns]
dd_df_religious = combined_df[selected_columns]
dd_df_religious

,time_slot,4,13,113,140,142,143,151,166,232,244
0,2021-01-01 00:00:00,1.0,1.0,6.0,3.0,2.0,2.0,3.0,3.0,0.0,0.0
1,2021-01-01 00:15:00,1.0,1.0,1.0,6.0,8.0,2.0,8.0,1.0,0.0,0.0
2,2021-01-01 00:30:00,1.0,0.0,5.0,5.0,17.0,4.0,6.0,4.0,1.0,2.0
3,2021-01-01 00:45:00,0.0,1.0,5.0,5.0,10.0,3.0,5.0,1.0,1.0,0.0
4,2021-01-01 01:00:00,0.0,0.0,2.0,3.0,18.0,2.0,8.0,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,2.0,5.0,36.0,11.0,33.0,5.0,3.0,4.0,1.0,0.0
99248,2023-10-31 23:00:00,3.0,3.0,35.0,14.0,38.0,1.0,4.0,1.0,2.0,0.0
99249,2023-10-31 23:15:00,0.0,1.0,47.0,10.0,118.0,4.0,2.0,2.0,2.0,1.0
99250,2023-10-31 23:30:00,1.0,3.0,44.0,7.0,43.0,3.0,1.0,0.0,0.0,1.0


In [116]:
dd_df_religious.to_csv('dd_df_religious.csv', index=False, header=True)

In [117]:
# total fair amount df for zones_with_school
total_fair_df_religious = combined_total_fair_df[selected_columns]
total_fair_df_religious

,time_slot,4,13,113,140,142,143,151,166,232,244
0,2021-01-01 00:00:00,22.800000,28.300000,21.086667,14.453333,17.900000,21.050000,11.966667,11.616667,0.000000,0.00
1,2021-01-01 00:15:00,27.800000,23.800000,22.880000,16.840000,15.125000,15.650000,14.907500,3.800000,0.000000,0.00
2,2021-01-01 00:30:00,13.800000,0.000000,18.234000,13.308000,19.153529,16.182500,13.785000,10.275000,29.800000,18.17
3,2021-01-01 00:45:00,0.000000,21.800000,26.010000,17.304000,14.283000,16.313333,4.680000,11.760000,17.750000,0.00
4,2021-01-01 01:00:00,0.000000,0.000000,11.075000,17.733333,18.282222,10.840000,10.550000,27.853333,26.163333,0.00
...,...,...,...,...,...,...,...,...,...,...,...
99247,2023-10-31 22:45:00,33.950000,43.092000,29.217222,24.420000,25.409697,19.140000,22.930000,25.317500,42.200000,0.00
99248,2023-10-31 23:00:00,20.256667,32.426667,28.443714,19.025000,23.725789,14.640000,19.140000,23.040000,20.760000,0.00
99249,2023-10-31 23:15:00,0.000000,31.440000,26.004468,21.360000,23.156356,32.297500,19.850000,12.210000,33.960000,22.55
99250,2023-10-31 23:30:00,66.880000,19.006667,25.276136,18.872857,33.357907,26.420000,25.800000,0.000000,0.000000,43.20


In [118]:
total_fair_df_religious.to_csv('total_fair_df_religious.csv', index=False, header=True)

### By Geographical Location

In [12]:
a = pd.read_csv('combined_dd_ss_raw_feat_data (1).csv')
a

,time_idx,datetime,group,target,inflow_lag168,inflow_lag336,outflow_lag168,outflow_lag336,inflow,inflow_diff1_per1,...,outflow_lag48__skewness,outflow_lag48__kurtosis,outflow_lag48__count_above_mean,outflow_lag48__count_below_mean,outflow_lag48__autocorrelation__lag_1,outflow_lag48__autocorrelation__lag_2,outflow_lag48__autocorrelation__lag_3,outflow_lag48__partial_autocorrelation__lag_1,outflow_lag48__partial_autocorrelation__lag_2,outflow_lag48__partial_autocorrelation__lag_3
0,336,2021-01-15 00:00:00,4,0.0,1.0,8.0,1.0,3.0,1.0,-5.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
1,337,2021-01-15 01:00:00,4,0.0,1.0,13.0,2.0,1.0,0.0,-1.0,...,0.108234,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
2,338,2021-01-15 02:00:00,4,0.0,3.0,9.0,0.0,2.0,1.0,1.0,...,0.000000,-0.981927,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
3,339,2021-01-15 03:00:00,4,0.0,0.0,5.0,0.0,3.0,0.0,-1.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
4,340,2021-01-15 04:00:00,4,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.851991,0.684898,0.497071,0.852000,-0.174618,-0.120363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390739,23920,2023-09-24 16:00:00,88,1.0,11.0,18.0,15.0,12.0,0.0,-1.0,...,0.740680,-0.536717,12.0,13.0,0.460025,0.325215,0.391421,0.460025,0.144084,0.255292
390740,23921,2023-09-24 17:00:00,88,0.0,21.0,22.0,14.0,13.0,0.0,0.0,...,0.753146,-0.583458,12.0,13.0,0.458333,0.256320,0.337209,0.458333,0.058550,0.253769
390741,23922,2023-09-24 18:00:00,88,0.0,19.0,10.0,10.0,12.0,0.0,0.0,...,0.876505,-0.154560,12.0,13.0,0.435278,0.227296,0.221336,0.435278,0.046673,0.131930
390742,23923,2023-09-24 19:00:00,88,0.0,14.0,22.0,7.0,11.0,0.0,0.0,...,0.958488,0.086961,12.0,13.0,0.418712,0.126411,0.260109,0.418712,-0.059305,0.278507
